<a href="https://colab.research.google.com/github/Nikhitha-S-Pavan/Deep-learning-examples-using-keras/blob/main/colorclassification_acc%3E80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -uq "/content/drive/My Drive/kaggle_dataset/colorclassification.zip" -d "/content/drive/My Drive/kaggle_dataset/colorclassification/"

In [ ]:
import cv2 
im = cv2.imread("/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/resized/yellow/5.jpg")
im.shape

In [24]:
# Create directories and subdirectories to save resized images
import glob
import os
import shutil

for dir in glob.glob("/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/ColorClassification/*/*"):
  split= dir.split("/")
  path_folder = os.path.join("/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/resized/",split[8])
  if os.path.exists(path_folder):
    continue
  else:
    os.makedirs(path_folder)

In [25]:
import os
import glob
os.chdir("/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/ColorClassification")
labels = os.listdir()
print(labels)

['Black', 'Blue', 'Brown', 'Green', 'Violet', 'White', 'mask_black', 'mask_blue', 'mask_brown', 'mask_green', 'mask_orange', 'mask_red', 'mask_test', 'mask_violet', 'mask_white', 'mask_yellow', 'orange', 'red', 'yellow']


In [ ]:

resize_path_train = "/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/resized/"  
for idx, value in enumerate(labels):
    for im in glob.glob(os.path.join("/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/ColorClassification",value ,'*')):
      img  = cv2.imread(im)
      width = 300
      height = 300
      dim = (width, height)
      # resize image
      resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
      cv2.imwrite(os.path.join(resize_path_train, value, im.split("/")[-1]), resized)
      print(os.path.join(resize_path_train, value, im.split("/")[-1]))

In [28]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [78]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                   horizontal_flip = True,
                                   
                                   )

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/ColorClassification/',
                                                 target_size = (224,224),
                                                 batch_size =1,
                                                 class_mode = 'categorical',
                                                 subset="training")


test_set = train_datagen.flow_from_directory('/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/ColorClassification/',
                                            target_size = (224,224),
                                            batch_size = 2,
                                            class_mode = 'categorical',
                                            subset="validation")

Found 266 images belonging to 19 classes.
Found 63 images belonging to 19 classes.


In [84]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=64,padding="same",kernel_size=3, activation='relu', input_shape=[224,224, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer


# Adding a third convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=19, activation='softmax'))

In [85]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [86]:
cnn.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 112, 112, 32)      18464     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 100352)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               12845184  
_________________________________________________________________
dense_23 (Dense)             (None, 19)              

In [88]:
history=cnn.fit_generator(
    training_set,
    epochs = 100,
    validation_data = test_set,
)

Epoch 1/100
  5/266 [..............................] - ETA: 3s - loss: 1.2170 - accuracy: 0.4000    

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


266/266 [==============================] - 8s 29ms/step - loss: 0.9841 - accuracy: 0.6617 - val_loss: 3.6245 - val_accuracy: 0.2063
Epoch 2/100
266/266 [==============================] - 7s 28ms/step - loss: 0.8963 - accuracy: 0.6692 - val_loss: 3.6677 - val_accuracy: 0.1905
Epoch 3/100
266/266 [==============================] - 7s 28ms/step - loss: 0.8059 - accuracy: 0.6955 - val_loss: 3.4977 - val_accuracy: 0.2540
Epoch 4/100
266/266 [==============================] - 8s 28ms/step - loss: 1.0274 - accuracy: 0.6805 - val_loss: 3.0572 - val_accuracy: 0.2063
Epoch 5/100
266/266 [==============================] - 8s 28ms/step - loss: 0.7467 - accuracy: 0.7143 - val_loss: 3.6579 - val_accuracy: 0.2381
Epoch 6/100
266/266 [==============================] - 7s 28ms/step - loss: 0.7942 - accuracy: 0.7143 - val_loss: 3.9445 - val_accuracy: 0.2222
Epoch 7/100
266/266 [==============================] - 7s 28ms/step - loss: 0.7506 - accuracy: 0.7331 - val_loss: 3.7260 - val_accuracy: 0.2063
Epoc

In [89]:
cnn.save("/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/ColorClassification/color_classification.h5")

In [96]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
# load and prepare the image
def load_image(filename):
  
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img
 


In [97]:
from keras import models    
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/test/testimg/96.jpg')
	# load model
	model = models.load_model("/content/drive/My Drive/kaggle_dataset/colorclassification/colorclassification/ColorClassification/color_classification.h5")
	# predict the class
	result = model.predict(img)
	print(result[0])
  
 
# entry point, run the example
run_example()

[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 9.2881197e-07 0.0000000e+00 9.9999905e-01]
